In [1]:
from transformers import pipeline
from datasets import load_dataset
import jsonlines
from tqdm.auto import tqdm

In [7]:
data_name = "SetFit/bbc-news"
dataset = load_dataset(data_name, split="train")

Using custom data configuration SetFit--bbc-news-dbeb222bfdd4d6ed
Found cached dataset json (C:/Users/heegyukim/.cache/huggingface/datasets/SetFit___json/SetFit--bbc-news-dbeb222bfdd4d6ed/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


In [3]:
device = 'cuda:0'
model = pipeline('text-generation', 'gpt2', device=device)

In [4]:
classifier = pipeline('text-classification', model='abhishek/autonlp-bbc-news-classification-37229289', device=device)

In [8]:
filename = data_name.replace("/", "__")
with jsonlines.open(f"data/gpt2-{filename}.jsonl", 'w') as f:
    for item in tqdm(dataset):
        text = item['text'].strip()
        if len(text) >= 512:
            text = text[-512:]
        gens = model(text, max_new_tokens=32, num_return_sequences=5)
        gens = [g['generated_text'][len(text):] for g in gens]
        item['generation'] = gens
        item['prediction'] = classifier(gens)
        f.write(item)
        # print(text)
        # print(item['label'])
        # print(gens)
        # print(classifier(gens))

100%|██████████| 1225/1225 [08:22<00:00,  2.44it/s]


In [25]:
import jsonlines
from collections import defaultdict

label2id = {
  "business": 0,
  "entertainment": 1,
  "politics": 2,
  "sport": 3,
  "tech": 4
}
fix_label = {
  2: 3,
  4: 2,
  3: 1,
  1: 0,
  0: 4
}
total = defaultdict(lambda: 0)
correct = defaultdict(lambda: 0)

with jsonlines.open("data/gpt2-SetFit__bbc-news.jsonl") as f:
  for item in tqdm(f):
    label = fix_label[item['label']]
    preds = item['prediction']

    total[label] += 5

    for p in preds:
      if label2id[p['label']] == label:
        correct[label] += 1


1225it [00:00, 58334.91it/s]


In [26]:
for k in total.keys():
    print(k, correct[k] / total[k])

3 0.6967272727272728
0 0.8174825174825174
1 0.799047619047619
4 0.6396226415094339
2 0.5884297520661157


In [27]:
correct

defaultdict(<function __main__.<lambda>()>,
            {3: 958, 0: 1169, 1: 839, 4: 678, 2: 712})